In [11]:
%pip install chardet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset

In [13]:
# Load dataset with detected encoding
df = pd.read_csv('all-data-engineering-quiz-utf.csv')

In [14]:
# Combine relevant text fields for input, joining reasoning fields if they are arrays
df['input_text'] = (
    df['Question'] + " " +
    df['Student Answer'] + " " +
    df['Ideal Answer'] + 
    " Grammar Reasoning: " + df['Grammar Reasoning'].apply(lambda x: ' | '.join(eval(x)) if isinstance(x, str) and x.startswith('[') else str(x)) + 
    " Structure Reasoning: " + df['Structure Reasoning'].apply(lambda x: ' | '.join(eval(x)) if isinstance(x, str) and x.startswith('[') else str(x))
)

# Define target columns
df['content_relevancy_score'] = df['Content Relevancy Score'].astype(float)
df['grammar_score'] = df['Grammar Score'].astype(float)
df['structure_score'] = df['Structure Score'].astype(float)

# Scale scores to range from 0 to 1 (optional for normalized regression)
df['content_relevancy_score'] /= 3  # Normalize content relevancy score to 0-1
df['grammar_score'] /= 5            # Normalize grammar score to 0-1
df['structure_score'] /= 5          # Normalize structure score to 0-1

# Train-test split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)



In [15]:
from transformers import BertConfig

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
config = BertConfig.from_pretrained('bert-base-uncased', hidden_dropout_prob=0.3)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Custom Dataset Class
class EssayDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['input_text']
        content_score = self.dataframe.iloc[idx]['content_relevancy_score']
        grammar_score = self.dataframe.iloc[idx]['grammar_score']
        
        # Tokenize input text
        inputs = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Combine scores into a single tensor for regression
        labels = torch.tensor([content_score, grammar_score], dtype=torch.float)
        
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': labels
        }

# Create datasets
max_length = 512
train_dataset = EssayDataset(train_df, tokenizer, max_length)
val_dataset = EssayDataset(val_df, tokenizer, max_length)


In [17]:
# Define custom metrics function for evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze()
    mse = mean_squared_error(labels, predictions, squared=False)
    return {'rmse': mse}

from transformers import AdamW, Trainer, TrainingArguments

# Adjusted Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,              # Lower learning rate
    per_device_train_batch_size=8,   # Reduce batch size if needed for memory
    per_device_eval_batch_size=8,
    num_train_epochs=50,             # Increase epochs
    weight_decay=0.02,               # Increased weight decay
    load_best_model_at_end=True,
    metric_for_best_model="rmse",
    greater_is_better=False,
    logging_dir='./logs',
    logging_steps=10,
    gradient_accumulation_steps=2    # Simulate larger batch size
)

# Initialize Trainer with custom optimizer if necessary
optimizer = AdamW(model.parameters(), lr=5e-6, weight_decay=0.02)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)     # Custom optimizer if needed
)

# Fine-tune the model
trainer.train()

d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  2%|▏         | 2/100 [01:08<55:36, 34.05s/it]d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  2%|▏         | 2/100 [01:21<55:36, 34.05s/it]

{'eval_loss': 0.7782740592956543, 'eval_rmse': 1.0445607900619507, 'eval_runtime': 5.2576, 'eval_samples_per_second': 1.902, 'eval_steps_per_second': 0.38, 'epoch': 0.8}


  5%|▌         | 5/100 [02:40<48:42, 30.76s/it]d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  5%|▌         | 5/100 [02:46<48:42, 30.76s/it]

{'eval_loss': 0.7293118238449097, 'eval_rmse': 0.896094799041748, 'eval_runtime': 5.7719, 'eval_samples_per_second': 1.733, 'eval_steps_per_second': 0.347, 'epoch': 2.0}


  7%|▋         | 7/100 [03:54<52:28, 33.86s/it]d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  7%|▋         | 7/100 [04:07<52:28, 33.86s/it]

...

d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

100%|██████████| 100/100 [1:14:22<00:00, 47.24s/it]

{'eval_loss': 0.5502617359161377, 'eval_rmse': 0.34550225734710693, 'eval_runtime': 8.9528, 'eval_samples_per_second': 1.117, 'eval_steps_per_second': 0.223, 'epoch': 40.0}


100%|██████████| 100/100 [1:14:25<00:00, 44.65s/it]

{'train_runtime': 4465.3285, 'train_samples_per_second': 0.403, 'train_steps_per_second': 0.022, 'train_loss': 1.1996148586273194, 'epoch': 40.0}


TrainOutput(global_step=100, training_loss=1.1996148586273194, metrics={'train_runtime': 4465.3285, 'train_samples_per_second': 0.403, 'train_steps_per_second': 0.022, 'total_flos': 378879919718400.0, 'train_loss': 1.1996148586273194, 'epoch': 40.0})

In [20]:
# Evaluate the model
metrics = trainer.evaluate()
print(metrics)

100%|██████████| 2/2 [00:01<00:00,  1.16it/s]d:\research_models\handwritten_ocr\handwritten_ocr\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
100%|██████████| 2/2 [00:01<00:00,  1.16it/s]

{'eval_loss': 0.5578422546386719, 'eval_rmse': 0.25804734230041504, 'eval_runtime': 8.6532, 'eval_samples_per_second': 1.156, 'eval_steps_per_second': 0.231, 'epoch': 40.0}


In [19]:
# Save the fine-tuned model
trainer.save_model('./essay_scoring_model')
tokenizer.save_pretrained('./essay_scoring_model')

('./essay_scoring_model\\tokenizer_config.json',
 './essay_scoring_model\\special_tokens_map.json',
 './essay_scoring_model\\vocab.txt',
 './essay_scoring_model\\added_tokens.json')